**Consolidated Monthly Rapport for DK**

This code generates the Monthly Rapport for DK

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [ ]:
# Define the file path for Shipments Out
# Replace the path with the actual path to your Shipments Consolidated file
shipments = pd.read_csv(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\Shipments_Consolidated.csv')

# Define the file path for Goods In
# Replace the path with the actual path to your Inbound Orderline file
inbound = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inbound Order Line.xlsx')

# Define the file path for Moved to Production
# Replace the path with the actual path to your Production file
production = pd.read_csv(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\Production_Consolidated.csv')

# Define the file path for Inbound from Production
# Replace the path with the actual path to your Inbound from Production file
inbound_prod = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inbound from Production.xlsx')

# Define the file path for Inventory Ultimo
# Replace the path with the actual path to your Inventory Consolidated file
inventory_skandflen = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inventory Skanderborg Flensborg.xlsx')
inventory_handewitt = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Final Inventory Handewitt.xlsx')

# Define the file path for Current Primo
# Replace the path with the actual path to your previous month's DK Rapport file
previous_rapport_DK = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) July 2025 Rapport.xlsx', sheet_name='DK')
# Replace the path with the actual path to your previous month's DE Rapport file
previous_rapport_DE = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) July 2025 Rapport.xlsx', sheet_name='DE')
# Replace the path with the actual path to your previous month's DE Rapport file
previous_rapport_TOT = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) July 2025 Rapport.xlsx', sheet_name='Total')

Data Transformation - Inventory

In [ ]:
# Cleanup files
# Cleanup the Inventory files
inventory_skandflen['Område'] = np.where(inventory_skandflen['Område'].isin(['S2', 'S4']), 'DK', np.where(inventory_skandflen['Område'] == 'F1', 'DE', inventory_skandflen['Område']))
inventory_handewitt['Område'] = 'DE'

# Aggregate the total Mængde per Varenummer and Område
inventory_skandflen_grouped = inventory_skandflen.groupby(['Varenummer', 'Område'], as_index=False)['Mængde'].sum()

# Remove #FP from Varenummer in inventory_skandflen_grouped
inventory_skandflen_grouped['Varenummer'] = inventory_skandflen_grouped['Varenummer'].str.replace('#FP', '', regex=False)

,Varenummer,Område,Mængde
0,###-2XL-B#FP,DK,6
1,###-2XL-W#FP,DK,8
2,###-L-B#FP,DK,14
3,###-L-W#FP,DK,4
4,###-M-B#FP,DK,11


In [14]:
# Aggregate the total On Hand per SKU and Område from Handewitt
inventory_handewitt_grouped = inventory_handewitt.groupby(['SKU', 'Område'], as_index=False)['On Hand'].sum()
inventory_handewitt_grouped.rename(columns={'SKU': 'Varenummer', 'On Hand': 'Mængde'}, inplace=True)
inventory_handewitt_grouped.head()


,Varenummer,Område,Mængde
0,6429,DE,0
1,6430,DE,0
2,6431,DE,0
3,6432,DE,0
4,6436,DE,0


In [ ]:
# Merge both inventory dataframes
final_inventory = pd.concat([inventory_skandflen_grouped, inventory_handewitt_grouped], ignore_index=True)
final_inventory = final_inventory.groupby(['Varenummer', 'Område'], as_index=False)['Mængde'].sum()
final_inventory.head()

,Varenummer,Område,Mængde
0,6429,DE,0
1,6430,DE,0
2,6431,DE,0
3,6432,DE,0
4,6436,DE,0


Data Transformation - Aggregations

In [ ]:
# Remove "#FP" from SKU in July Rapport files
previous_rapport_DK['SKU'] = previous_rapport_DK['SKU'].str.replace('#FP', '', regex=False)
previous_rapport_DE['SKU'] = previous_rapport_DE['SKU'].str.replace('#FP', '', regex=False)
previous_rapport_TOT['SKU'] = previous_rapport_TOT['SKU'].str.replace('#FP', '', regex=False)

# Extract the Current DK Primo using the previous month's "Inventroy ULTIMO"
ULTIMO_DK = previous_rapport_DK.groupby('SKU', as_index=False)['Inventroy ULTIMO'].sum()

# Extract the Current DE Primo using the previous month's "Inventroy ULTIMO"
ULTIMO_DE = previous_rapport_DE.groupby('SKU', as_index=False)['Inventroy ULTIMO'].sum()
ULTIMO_DE.head()

# Extract the Current TOT Primo using the previous month's "Inventroy ULTIMO"


,SKU,Inventroy ULTIMO
0,###-2XL-B,10
1,###-2XL-W,8
2,###-L-B,19
3,###-L-W,19
4,###-M-B,20


In [32]:
# Extract the Current DE Primo using the previous month's "Inventroy ULTIMO"
ULTIMO_DE = previous_rapport_DE.groupby('SKU', as_index=False)['Inventroy ULTIMO'].sum()
ULTIMO_DE.head().sort_values(by='Inventroy ULTIMO')
ULTIMO_DE.to_csv('ULTIMO_DE.csv', index=False)

## Separator ##
Last we left off was creating the Inventory ULTIMO into our script.

In [ ]:
# --- Aggregations (SUMIF-by-SKU) ---
goods_in = (
    inbound.groupby("Varenummer", dropna=False)["Mængde modtaget"]
    .sum()
    .reset_index()
    .rename(columns={"Varenummer": "SKU", "Mængde modtaget": "Goods In"})
)

moved_to_production = (
    production.groupby("ItemNumber", dropna=False)["QuantityProduced"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityProduced": "Moved to Production"})
)

inbound_from_production = (
    inbound_prod.groupby("ItemNumber", dropna=False)["QuantityPacked"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityPacked": "Inbound from Production"})
)

shipments_out = (
    shipments.groupby("ItemNumber", dropna=False)["QuantityPacked"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityPacked": "Shipments Out"})
)

# --- Merge into one report ---
report = (
    goods_in
    .merge(moved_to_production, on="SKU", how="outer")
    .merge(inbound_from_production, on="SKU", how="outer")
    .merge(shipments_out, on="SKU", how="outer")
)

## Separator ##

In [ ]:
# Groupby to create summary tables
goods_in = inbound.groupby("Varenummer")["Mængde modtaget"].sum().reset_index()
moved_to_prod = production.groupby("ItemNumber")["QuantityProduced"].sum().reset_index()
inbound_from_prod = inbound_prod.groupby("ItemNumber")["QuantityPacked"].sum().reset_index()
shipments_out = shipments.groupby("ItemNumber")["QuantityPacked"].sum().reset_index()

In [ ]:
# Merge all summary tables into a single report

report = goods_in.merge(moved_to_prod, on="SKU", how="outer") \
                 .merge(inbound_from_prod, on="SKU", how="outer") \
                 .merge(shipments_out, on="SKU", how="outer")

In [5]:
import pandas as pd
from pathlib import Path

# --- File paths (edit if needed) ---
base = Path(r"C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup")

shipments_path = base / "Shipments_Consolidated.csv"
inbound_order_path = base / "2025-09-02T03_44_52_230_inboundOrderLine (3).xlsx"
production_path = base / "Production_Consolidated.csv"
inbound_from_prod_path = base / "(ITA) Inbound from Production.xlsx"

# --- Load files ---
shipments = pd.read_csv(shipments_path)

# inbound order line (Excel, single sheet: 'inboundOrderLine')
inbound = pd.read_excel(inbound_order_path, sheet_name="inboundOrderLine", engine="openpyxl")

# production consolidated (CSV)
production = pd.read_csv(production_path)

# inbound from production (Excel, two sheets: DK + DE) -> concat
inbound_prod_dk = pd.read_excel(inbound_from_prod_path, sheet_name="Aug25 Prod Order-DK", engine="openpyxl")
inbound_prod_de = pd.read_excel(inbound_from_prod_path, sheet_name="Aug25 Prod Order - DE", engine="openpyxl")
inbound_prod = pd.concat([inbound_prod_dk, inbound_prod_de], ignore_index=True)

# --- Aggregations (SUMIF-by-SKU) ---
goods_in = (
    inbound.groupby("Varenummer", dropna=False)["Mængde modtaget"]
    .sum()
    .reset_index()
    .rename(columns={"Varenummer": "SKU", "Mængde modtaget": "Goods In"})
)

moved_to_production = (
    production.groupby("ItemNumber", dropna=False)["QuantityProduced"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityProduced": "Moved to Production"})
)

inbound_from_production = (
    inbound_prod.groupby("ItemNumber", dropna=False)["QuantityPacked"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityPacked": "Inbound from Production"})
)

shipments_out = (
    shipments.groupby("ItemNumber", dropna=False)["QuantityPacked"]
    .sum()
    .reset_index()
    .rename(columns={"ItemNumber": "SKU", "QuantityPacked": "Shipments Out"})
)

# --- Merge into one report ---
report = (
    goods_in
    .merge(moved_to_production, on="SKU", how="outer")
    .merge(inbound_from_production, on="SKU", how="outer")
    .merge(shipments_out, on="SKU", how="outer")
)

# Fill missing numeric values
for col in ["Goods In", "Moved to Production", "Inbound from Production", "Shipments Out"]:
    if col in report.columns:
        report[col] = report[col].fillna(0)

# Optional extra columns (remove if you only want the four sums)
report["In Production"] = report["Moved to Production"] - report["Inbound from Production"]  # WIP proxy
report["Serial no."] = pd.NA   # placeholder
report["Variance"] = 0         # placeholder
report["Inventory ULTIMO"] = report["Goods In"] + report["Inbound from Production"] - report["Shipments Out"]

# Choose/Order columns
cols = [
    "SKU", "Goods In", "Moved to Production", "In Production",
    "Inbound from Production", "Shipments Out", "Serial no.",
    "Variance", "Inventory ULTIMO"
]
report = report[[c for c in cols if c in report.columns]].sort_values("SKU", kind="stable")

# Save
out_path = base / "SKU_Flow_Report.csv"
report.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"Saved to: {out_path}")


Saved to: C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\SKU_Flow_Report.csv
